### Importazioni

In [10]:
import json
import requests
import pandas as pd
import torch
from tqdm import tqdm
from dotenv import dotenv_values
from pymongo import MongoClient
from torch import nn
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import pipeline

In [11]:
with open('UMBRIA - DICTIONARIES.txt', 'r') as file:
    contenuto = file.read()

# Decodifico il contenuto JSON in un dizionario Python
dizionario = json.loads(contenuto)

# Lista per le coordinate
coordinate = []
titoli = []

# Estraggo le longitudini e latitudini e le aggiungo alla lista coordinate
for elemento in dizionario.values():
    latitudine = elemento.get('lat')
    longitudine = elemento.get('lon')
    titolo = elemento.get('title')
    if latitudine is not None and longitudine is not None:
        coordinate.append((latitudine, longitudine))
        titoli.append(titolo)

print(str(len(coordinate)) + " POI")
print(titoli)

FileNotFoundError: [Errno 2] No such file or directory: 'UMBRIA - DICTIONARIES.txt'

## Estraggo le coordinate dai POI

In [ ]:
# Apro il file e leggo il contenuto
with open(r'C:\Users\franc\Desktop\PROJECTS\PROJECTS\RASTA\POI\DICTIONARYTOTALE.txt', 'r') as file:
    contenuto = file.read()


In [ ]:
# Decodifico il contenuto JSON in un dizionario Python
dizionario = json.loads(contenuto)

# Lista per le coordinate
coordinate = []

# Estraggo le longitudini e latitudini e le aggiungo alla lista coordinate
for elemento in dizionario.values():
    latitudine = elemento.get('lat')
    longitudine = elemento.get('lon')
    if latitudine is not None and longitudine is not None:
        coordinate.append((latitudine, longitudine))

# Stampo la lista delle coordinate
print(coordinate)

[(42.68827, 12.549764), (42.683333, 12.55), (42.690376, 12.54649), (42.691138, 12.54658), (42.69067, 12.546327), (42.70388888888889, 12.574444444444444), (42.698055555555555, 12.531388888888888), (42.673620035730664, 12.5873151), (42.673169, 12.603323), (42.656894, 12.536004), (42.7100669, 12.4951428), (42.70867, 12.48504), (42.664167, 12.508611), (42.71342, 12.50648), (42.711472, 12.611841), (42.708, 12.621), (42.65, 12.55), (42.64827, 12.637932), (42.73033, 12.58681), (42.7847, 12.0356), (42.786111, 12.037778), (42.772684, 12.039017), (42.771667, 12.018611), (42.804302, 12.071065), (42.75527777777778, 12.001666666666667), (42.80222222222222, 11.983888888888888), (42.833333, 12.066667), (42.835532, 12.065821), (42.816667, 11.966667), (42.755278, 11.945833), (42.7919, 11.9175), (42.7805, 11.9067), (42.587742, 12.295626), (42.583333, 12.3), (42.591907, 12.25983), (42.574862, 12.32824), (42.616667, 12.3), (42.566667, 12.333333), (42.62388, 12.29671), (42.621667, 12.254444), (42.546496, 1

In [ ]:
print(str(len(coordinate)) + " POI")

3141 POI


In [ ]:
poche_coordinate = coordinate[1000:1200]

print(poche_coordinate)
print(coordinate[1200])

[(42.81666666666667, 11.883333333333333), (42.55, 12.42), (42.55255, 12.41634), (42.54218, 12.45576), (42.5326, 12.76345), (42.5441, 12.7211), (43.04406, 12.62069), (43.045558, 12.612161), (43.030277777777776, 12.651111111111112), (42.736667, 12.369444), (42.72878, 13.15612), (42.69347, 13.17673), (42.70927, 13.18948), (42.72224, 13.19937), (42.9475, 12.455277777777777), (43.134932, 12.346319), (43.37416666666667, 12.78888888888889), (43.026389, 12.89), (43.307271, 12.845069), (42.466667, 12.3125), (43.3655412, 12.5826346), (43.365278, 12.036111), (43.42113, 11.97621), (43.42481, 11.977058), (43.422402, 11.974812), (43.359167, 11.997222), (43.349722, 12.034444), (43.350833, 12.016667), (43.46372, 12.04123), (43.463059, 12.019941), (43.465708, 12.039932), (43.38166666666667, 12.223611111111111), (43.40039, 12.23761), (43.381188, 12.225313), (43.37985, 12.23896), (43.370556, 12.238056), (43.38544, 12.13118), (43.34285, 12.082697), (42.514066, 12.520792), (42.51208333333334, 12.5214444444

In [ ]:
print(len(poche_coordinate))

200


## Recupero la chiave API

In [ ]:
dotenv = dotenv_values()
api_key = dotenv.get("API_KEY");

## Estraggo i POI presenti su TripAdvisor

In [ ]:
# Creo un insieme per memorizzare gli ID già presenti nel file
id_presenti = set()

# Controllo se il file esiste
import os
# Se il file esiste, leggo il contenuto attuale del file e memorizzo gli ID
if os.path.exists('poi.txt'):
    with open('poi.txt', 'r', encoding='latin-1') as file:
        for line in file:
            # Divido la riga usando ': ' come separatore solo alla prima occorrenza
            id, resto = line.strip().split(': ', 1)
            # Divido il resto della riga usando ', ' come separatore per ottenere il nome del POI
            nome = resto.split(', ')[0]
            id_presenti.add(id)
   
# Creo la lista vuota di POI da salvare
poi = []

# Mi salvo nella lista l'id e il nome del POI solo se l'id non è già presente
for coordinate in poche_coordinate:
    url = "https://api.content.tripadvisor.com/api/v1/location/nearby_search?latLong=" + str(coordinate[0]) + "%2C" + str(coordinate[1]) + "&key=" + api_key + "&category=attractions&language=it"
    headers = {"accept": "application/json"}
    response = requests.get(url, headers=headers)
   
    POI = response.json()

    # Estraggo l'id e il nome di ciascun POI e li aggiungo alla lista di POI se l'id non è già presente
    for elemento in POI['data']:
        nome = elemento.get('name')
        id = elemento.get('location_id')
        citta = elemento.get('address_obj')
        if nome is not None and id is not None and id not in id_presenti:
            poi.append((id, nome, citta))
            # Aggiungo l'id all'insieme degli ID presenti
            id_presenti.add(id)

# Apro il file in modalità scrittura in modalità append
with open('poi.txt', 'a', encoding='utf-8') as file:
    # Itero sulla lista di POI e scrivo ciascun POI nel file solo se l'id non è già presente
    for id, nome, citta in poi:
        file.write(f"{id}: {nome}, {citta}\n")

print(poi)
    

[('19565006', 'Chiesa della Madonna della Quercia', {'street1': 'Strada Provinciale 51 Trevinianese', 'street2': '', 'city': 'Trevinano', 'state': 'Provincia di Viterbo', 'country': 'Italia', 'address_string': 'Strada Provinciale 51 Trevinianese, Trevinano, Acquapendente Italia'}), ('23430519', 'Centro Storico Di Roccella Ionica', {'street1': 'Centro Storico', 'street2': '', 'city': 'Roccella Ionica', 'state': 'Provincia di Reggio Calabria', 'country': 'Italia', 'address_string': 'Centro Storico, Roccella Ionica Italia'}), ('21330796', 'Chiesa Di Santa Chiara', {'street1': 'Centro Storico', 'street2': '', 'city': 'Stilo', 'state': 'Provincia di Reggio Calabria', 'country': 'Italia', 'address_string': 'Centro Storico, Stilo Italia'}), ('4498581', 'Parco Castel Pagano', {'street1': 'Strada delle Rote 2', 'street2': 'Porchiano del Monte', 'city': 'Amelia', 'state': 'Provincia di Terni', 'country': 'Italia', 'postalcode': '05022', 'address_string': 'Strada delle Rote 2 Porchiano del Monte,

In [ ]:
print(len(poi))

207


### Creo il poi_dict

In [ ]:
# Inizializzo un dizionario vuoto per memorizzare gli ID e i nomi dei POI
poi_dict = {}

# Leggo il contenuto del file riga per riga
with open('poi.txt', 'r', encoding='latin-1') as file:
    for line in file:
        # Divido la riga usando ': ' come separatore solo alla prima occorrenza
        id, resto = line.strip().split(': ', 1)
        # Divido il resto della riga usando ', ' come separatore per ottenere il nome del POI
        nome = resto.split(', ')[0]
        # Aggiungo l'id e il nome al dizionario
        poi_dict[id] = nome

# Ora poi_dict conterrà gli ID come chiavi e i nomi dei POI come valori
print(poi_dict)


{'8666261': 'Palazzo Cesi', '13436625': 'Chiesa di Santa Cecilia', '21223808': 'Lago Di Arezzo', '19071050': 'Frantoio Oleario Palmadori Massimo', '2016179': 'Carsulae - Parco Archeologico', '19487544': 'Chiesa San Bernardino', '19485030': 'Chiesa di Santï¿½Apollinare', '21389509': 'Chiesa San Faustino', '17785314': "Pieve di Sant'Antonio Abate", '17780645': "Chiesa di Sant'Antonio Abate", '17781582': "Chiesa Di Sant'antonio Abate", '617225': "Velia's Cooking Style", '11851556': 'Bottega del Castello della Sale', '10384019': 'Cirulli - Aziende Agricole', '12673247': 'Castello di Madonna Antonia', '17780814': 'chiesa della S.S. Annunziata', '27514189': 'Parrucchiera Giulia', '13436641': 'Museo dei Cicli Geologici', '13436639': 'Chiesa di San Michele Arcangelo', '13436560': "Chiesa della Madonna dell'Acqua", '4828006': 'Borgo Antico Bar', '10727071': 'Chiesa Santa Maria Assunta', '26437754': 'Shiatsuqi di Natascia Minciarelli', '26628676': 'Le Fontane - Lavatoio pubblico', '17780615': 'A

In [ ]:
for elemento in poi_dict:
    print(elemento)

8666261
13436625
21223808
19071050
2016179
19487544
19485030
21389509
17785314
17780645
17781582
617225
11851556
10384019
12673247
17780814
27514189
13436641
13436639
13436560
4828006
10727071
26437754
26628676
17780615
23603031
26454141
23498616
17785149
23527414
17784926
8442796
6362879
3316846
10751519
6873086
13506803
23736060
5988820
13908621
8818259
15234702
19486021
19485904
21129953
15234713
7927400
19635998
12827750
5937150
13908574
26858943
26848269
10717566
23590891
24077147
21129964
19485106
15582334
15582326
12941174
15582304
12941300
20943988
4178733
15582352
9974289
14902761
10536774
15582310
17567725
10125575
21330794
21341165
23082361
21341156
23082363
15582323
21341167
21341166
4103928
20947702
10120122
7117226
1583158
23919722
21272422
23066524
23430908
21330795
21329540
4131906
23410936
19485109
15843151
10821335
6878472
12819225
10820447
10820435
10820353
19940269
19344024
10342709
23682565
25336273
24033005
3400132
1392223
20724089
20724098
11502819
13436659
66091

In [ ]:
len(poi_dict)

1816

### Salvo i POI in un dizionario JSON

In [ ]:
# salvo il dizionario in un file json
dizionario_json = "poi.json"

# Salvataggio del dizionario nel file JSON
with open(dizionario_json, "w") as file:
    json.dump(poi_dict, file)

## Recupero le recensioni relative ai POI estratti

In [ ]:
items_list = list(poi_dict.items())
print(items_list)

for elemento in items_list[1609:]:
    print(elemento[0])


[('8666261', 'Palazzo Cesi'), ('13436625', 'Chiesa di Santa Cecilia'), ('21223808', 'Lago Di Arezzo'), ('19071050', 'Frantoio Oleario Palmadori Massimo'), ('2016179', 'Carsulae - Parco Archeologico'), ('19487544', 'Chiesa San Bernardino'), ('19485030', 'Chiesa di Santï¿½Apollinare'), ('21389509', 'Chiesa San Faustino'), ('17785314', "Pieve di Sant'Antonio Abate"), ('17780645', "Chiesa di Sant'Antonio Abate"), ('17781582', "Chiesa Di Sant'antonio Abate"), ('617225', "Velia's Cooking Style"), ('11851556', 'Bottega del Castello della Sale'), ('10384019', 'Cirulli - Aziende Agricole'), ('12673247', 'Castello di Madonna Antonia'), ('17780814', 'chiesa della S.S. Annunziata'), ('27514189', 'Parrucchiera Giulia'), ('13436641', 'Museo dei Cicli Geologici'), ('13436639', 'Chiesa di San Michele Arcangelo'), ('13436560', "Chiesa della Madonna dell'Acqua"), ('4828006', 'Borgo Antico Bar'), ('10727071', 'Chiesa Santa Maria Assunta'), ('26437754', 'Shiatsuqi di Natascia Minciarelli'), ('26628676', '

### Creo il dict recensioni_per_location

In [ ]:
# Dizionario per memorizzare le recensioni per ogni location
recensioni_per_location = {}

items_list = list(poi_dict.items())
# Itero sui POI e recupera le recensioni
for elemento in items_list[1609:]:
    url = "https://api.content.tripadvisor.com/api/v1/location/" + elemento[0] + "/reviews?key=" + api_key + "&language=it"
    headers = {"accept": "application/json"}
    response = requests.get(url, headers=headers)
    REC = response.json()

    # Estraggo le recensioni di ciascun POI e le aggiungo al dizionario
    for elemento in REC['data']:
        id_rec = elemento.get('id')
        id_loc = elemento.get('location_id')
        rating = elemento.get('rating')
        recensione = elemento.get('text')

        # Se la location_id non è già presente nel dizionario, l'aggiungo come chiave con una lista vuota come valore
        if id_loc not in recensioni_per_location:
            recensioni_per_location[id_loc] = {}

        # Aggiungo la recensione al dizionario nidificato utilizzando l'id della recensione come chiave
        recensioni_per_location[id_loc][id_rec] = {'rating': rating, 'recensione': recensione}

# Scrivo il dizionario di recensioni su un file JSON
with open('recensioni_per_location1.json', 'w') as file:
    json.dump(recensioni_per_location, file)

In [ ]:
REC

{'data': [{'id': 772412137,
   'lang': 'it',
   'location_id': 19945921,
   'published_date': '2020-09-29T15:54:07Z',
   'rating': 4,
   'helpful_votes': 1,
   'rating_image_url': 'https://www.tripadvisor.com/img/cdsi/img2/ratings/traveler/s4.0-66827-5.svg',
   'url': 'https://www.tripadvisor.it/ShowUserReviews-g187905-d19945921-r772412137-Reviews-Sorella_Luna-Assisi_Province_of_Perugia_Umbria.html?m=66827#review772412137',
   'text': 'L’abbiamo scelto per la posizione strategica vicino alla cattedrale, l’abbiamo apprezzato per il fascino, la pulizia e la gentilezza dello staff. Di gran gusto la ristrutturazione. L’unico piccolo neo : la colazione salata non particolarmente varia.',
   'title': 'Posizione strategica ',
   'trip_type': 'Viaggio con amici',
   'travel_date': '2020-09-30',
   'user': {'username': 'marcoditorino',
    'user_location': {'id': '187855',
     'name': 'Turin, Province of Turin, Piedmont'},
    'avatar': {'thumbnail': 'https://media-cdn.tripadvisor.com/media/ph

In [ ]:
len(recensioni_per_location)

163

In [ ]:
recensioni_per_location

{23430519: {835565836: {'rating': 3,
   'recensione': 'Sicuramente è un centro storico ben tenuto, con piazze alberate e qualche bar e ristorante, ma non ci sono emergenze architettoniche degne di nota'},
  790908598: {'rating': 4,
   'recensione': 'Ciò che colpisce il visitatore è l’intenso profumo di gelsomino che si sente per le strade e le piazze di questo grazioso borgo ionico.\n\nRoccella, infatti, si trova sulla suggestiva Costa dei Gelsomini, che comprende oltre quaranta comuni della provincia di Reggio Calabria.\n\nNei tempi passati, i fiori di gelsomino venivano raccolti\u200b dalle gelsominaie e mandati in Francia per la preparazione dei profumi.\n\nIl centro storico della città si presenta molto ben curato, pieno di locali e di chiese purtroppo chiuse\n\n'}},
 10296598: {855661323: {'rating': 5,
   'recensione': 'Abbiamo prenotato un tour dell’Umbria con autonoleggio con conducente Maurizio. Professionalità e disponibilità massima. Un aneddoto simpatico e curioso per ogni l

In [ ]:
# Apro il file in modalità lettura
with open('recensioni_per_location.json', 'r') as file:
    recensioni = file.read();
    # Decodifico il contenuto JSON in un dizionario Python

#print(recensioni)
print(len(recensioni))
dizionario_recensioni = json.loads(recensioni)

1437306


In [ ]:
# recupero anche il dizionario con solo chiave univoca della location e il nome
with open('poi.json', 'r') as file:
    poi_json = file.read()
    dizionario_poi = json.loads(poi_json)

In [ ]:
for items in dizionario_poi.items():
    for idloc, recensione_info in dizionario_recensioni.items():
        recensione_info = {'POI': "nome_poi", **recensione_info}

In [ ]:
for id_rec, recensione_info in dizionario_recensioni.items():
    # Controllo se l'id della recensione corrisponde a un id del POI
    if id_rec in dizionario_poi:
        # Ottengo il nome del POI associato all'id del POI
        nome_poi = dizionario_poi[id_rec]
        # Aggiungo il nome del POI all'inizio della recensione
        recensione_info = {'POI': nome_poi, **recensione_info}
        # Aggiorno il dizionario delle recensioni con la recensione aggiornata
        dizionario_recensioni[id_rec] = recensione_info

# Ora dizionario_recensioni contiene il campo 'POI' aggiunto all'inizio di ciascuna recensione



In [ ]:
# Scrivo il dizionario di recensioni su un file JSON
with open('recensioni_complete.json', 'w') as file:
    json.dump(dizionario_recensioni, file)

In [ ]:
# Conto le recensioni estratte per ora

with open('recensioni_complete.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

# Inizializzo il contatore delle recensioni
numero_recensioni = 0

# Itero attraverso i dati ed estraggo le recensioni
for poi_id, poi_data in data.items():
    for recensione_id, recensione_info in poi_data.items():
        if isinstance(recensione_info, dict) and 'recensione' in recensione_info:
            recensione = recensione_info['recensione']
            print(recensione)  # Stampo la recensione 
            numero_recensioni += 1

# Stampa il numero totale di recensioni
print("Numero totale di recensioni:", numero_recensioni)


Abitando a pochi chilometri, ho scoperto questa attrazione grazie proprio a Tripadvisor. Purtroppo non c’è la benché minima indicazione, una recensione lo spiega ma Tripadvisor la traduce male. Provo a descriverla meglio: uscendo dalla superstrada (uscita Ripabianca), proseguire verso Collepepe; arrivati alla rotatoria prendere l’uscita verso la località “carceri” (prima uscita); proseguire e, appena passato il sottopassaggio, girare a sinistra su una strada bianca in salita. A quel punto proseguire fino a raggiungere l’ultima casa sulla sinistra, a destra c’è una strada che si insinua all’interno di un boschetto. Ebbene, sulla destra si troverà la prima cisterna (si vede una recinzione, basta fare il giro l’ingresso si trova dietro), la seconda si trova sulla sinistra sotto una tettoia. Purtroppo la prima è chiusa ma si intravede qualcosa, mentre la seconda è libera. 
Da queste cisterne (facenti parte di una villa romana ormai inesistente) si capisce quanta storia nascosta vi sia in I

### Salvo le recensioni con il rating in un file di testo

In [ ]:
# Carica il file JSON
with open('recensioni_complete.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

# Lista per contenere tutte le recensioni
recensioni = []

# Loop attraverso i dati estraendo le recensioni
for luogo in data.values():
    for recensione in luogo.values():
        recensioni.append(recensione)

# Scrivi le recensioni in un file di testo
with open('recensioni.txt', 'w', encoding='utf-8') as file:
    for recensione in recensioni:
        file.write(str(recensione) + '\n')

# Conta il numero di recensioni
numero_recensioni = len(recensioni)
print("Numero di recensioni estratte:", numero_recensioni)

Numero di recensioni estratte: 6715


### Creo un corpus


In [ ]:
# Carica il file JSON
with open('recensioni_complete.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

# Apri il file corpus.txt per scrittura
with open('corpus.txt', 'w', encoding='utf-8') as corpus_file:
    # Loop attraverso i POI
    for poi_id, poi_data in data.items():
        # Loop attraverso le recensioni di ogni POI
        for recensione_data in poi_data.values():
            # Controlla se esiste la chiave 'recensione'
            if 'recensione' in recensione_data:
                # Scrivi la recensione nel file corpus.txt
                recensione = recensione_data['recensione'].replace('\n', ' ')
                corpus_file.write(recensione + '\n')

In [ ]:
recensioni

['Palazzo Cesi',
 {'rating': 4,
  'recensione': 'Palazzo Cesi di Acquasparta é il principale monumento del borgo.\nVisitabile con un esiguo biglietto di 6 euro, venne fondato dal Duca Federico Cesi nel Seicento, e nelle sue sale venne ospitato Galileo Galilei.\nIl prospetto esterno, che dà sulla piazza principale, si presenta, nell’ordine inferiore, con uno splendido portale rinascimentale a tutto sesto, ornato da una cornice in conci bugnati.\nL’interno é impreziosito da un susseguirsi di sale che presentano splendide volte affrescate, in eccellente stato di conservazione, mentre altre sono ornate da elaborati soffitti lignei. Gli affreschi presenti, narrano vicende legate alla mitologia.\nLa struttura del palazzo é edificata su due piani, piano terra e primo piano, entrambi visitabili. Splendido é anche il loggiato del primo piano, ove si affacciano le stanze. Le varie sale che si susseguono nel percorso di visita sono  consequenziali tra loro, sia al piano terra che al primo, pertan

# Salvo nel database noSQL (MongoDB)


In [ ]:
# recupero la password del db
dotenv = dotenv_values()
db_password = dotenv.get("DB_PASSWORD");

In [ ]:
# Connessione al server MongoDB
client = MongoClient('mongodb+srv://fpulera:' + db_password + '@rasta.uukuftt.mongodb.net/')

In [ ]:
# Seleziona il database
db = client['RASTA']

# Seleziona la collezione
collection = db['poi']

In [ ]:

# Caricamento dei dati dal file JSON
with open('recensioni_complete.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

for key, value in data.items():
    db.poi.insert_one(value)

# Chiusura della connessione al server MongoDB
client.close()

In [ ]:
# Connessione al server MongoDB
client = MongoClient('mongodb+srv://fpulera:' + db_password + '@rasta.uukuftt.mongodb.net/')

# Seleziona il database
db = client['RASTA']

# Seleziona la collezione
collection = db['poi']

# Query per trovare tutte le recensioni 
cursor = collection.find()

# Iterazione sui risultati della query
for poi in cursor:
    # Accesso al nome del POI
    poi_name = poi['POI']
    
    # Accesso alle recensioni associate al POI
    reviews = poi.values()
    
    # Iterazione sulle recensioni
    for review in reviews:
        # Controllo se l'elemento è un dizionario (ovvero una recensione)
        if isinstance(review, dict):
            # Accesso al rating della recensione
            rating = review['rating']
            print(f"POI: {poi_name}, Rating: {rating}")
    
# Chiusura della connessione al server MongoDB
client.close()

POI: Palazzo Cesi, Rating: 4
POI: Palazzo Cesi, Rating: 4
POI: Palazzo Cesi, Rating: 4
POI: Palazzo Cesi, Rating: 5
POI: Palazzo Cesi, Rating: 5
POI: Chiesa di Santa Cecilia, Rating: 3
POI: Chiesa di Santa Cecilia, Rating: 5
POI: Chiesa di Santa Cecilia, Rating: 3
POI: Lago Di Arezzo, Rating: 4
POI: Frantoio Oleario Palmadori Massimo, Rating: 4
POI: Frantoio Oleario Palmadori Massimo, Rating: 5
POI: Frantoio Oleario Palmadori Massimo, Rating: 5
POI: Frantoio Oleario Palmadori Massimo, Rating: 5
POI: Carsulae - Parco Archeologico, Rating: 5
POI: Carsulae - Parco Archeologico, Rating: 5
POI: Carsulae - Parco Archeologico, Rating: 4
POI: Carsulae - Parco Archeologico, Rating: 5
POI: Carsulae - Parco Archeologico, Rating: 5
POI: Pieve di Sant'Antonio Abate, Rating: 5
POI: Chiesa di Sant'Antonio Abate, Rating: 5
POI: Chiesa di Sant'Antonio Abate, Rating: 5
POI: Chiesa Di Sant'antonio Abate, Rating: 4
POI: Chiesa Di Sant'antonio Abate, Rating: 5
POI: Velia's Cooking Style, Rating: 5
POI: Bot

# Sentiment - Italian BERT Sentiment model

https://huggingface.co/neuraly/bert-base-italian-cased-sentiment 

In [ ]:
df = pd.DataFrame(columns=['id', 'POI', 'rating', 'recensione'])
with open(r"C:\Users\franc\Desktop\PROJECTS\PROJECTS\RASTA\POI con recensioni\RASTA.poi.json", 'r', encoding='latin-1') as file:
    data = json.load(file)

# Append rows to the DataFrame
for a in data:
  id = a['_id']['$oid']
  poi = a['POI']

  for k in a.keys():
    if k != 'POI' and k!='_id':

      df = pd.concat([df, pd.DataFrame([{'id':id, 'POI': poi, 'rating':a[k]['rating'], 'recensione': a[k]['recensione']}])], ignore_index=True)

In [ ]:
df

,id,POI,rating,recensione
0,66374440e18e1c186ce07e35,Palazzo Cesi,5,"Un palazzo appena restaurato, una ricca docume..."
1,66374440e18e1c186ce07e35,Palazzo Cesi,5,Cinquecentesca dimora di una tra le famiglie i...
2,66374440e18e1c186ce07e35,Palazzo Cesi,4,L'imponente palazzo in stile rinascimentale Ã¨...
3,66374440e18e1c186ce07e35,Palazzo Cesi,4,Particolari elogi vanno per il recupero e cons...
4,66374440e18e1c186ce07e35,Palazzo Cesi,4,Palazzo Cesi di Acquasparta Ã© il principale m...
...,...,...,...,...
10039,663c8b16ec1e0d04a559c75e,Cacio Pepe e,5,Ci siamo trovati di passaggio dopo aver visita...
10040,663c8b16ec1e0d04a559c75f,Annette Jolie,5,"Girando per Assisi ,ho trovato un negozio fina..."
10041,663c8b16ec1e0d04a559c75f,Annette Jolie,5,A pochi passi da Piazza del Comune abbiamo sco...
10042,663c8b16ec1e0d04a559c760,Sorella Luna,5,Negozio posto in una via secondaria ma dove va...


In [ ]:
df.to_excel(r'C:\Users\franc\Desktop\PROJECTS\PROJECTS\RASTA\POI con recensioni\POI.xlsx', index=False)

In [ ]:
# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("neuraly/bert-base-italian-cased-sentiment")
# Load the model, use .cuda() to load it on the GPU
model = AutoModelForSequenceClassification.from_pretrained("neuraly/bert-base-italian-cased-sentiment")

c:\Users\franc\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


1. Estrae la recensione dalla colonna recensione.
2. Codifica la recensione utilizzando un tokenizer, parte di un modello di linguaggio preaddestrato.
3. Trasforma la sequenza codificata in un tensore e lo prepara per l'input al modello.
4. Passa il tensore al modello per ottenere i logit, che sono le uscite del modello prima di applicare la funzione di attivazione.
5. Applica una funzione di softmax sui logit per ottenere le probabilità associate a ciascuna classe (probabilità negative, neutre e positive).
6. Memorizza le probabilità negative, neutre e positive ottenute nella tabella DataFrame df.
7. Determina la classe di sentimento prevista utilizzando la probabilità massima tra negative, neutre e positive.
8. Memorizza la classe di sentimento prevista nella tabella DataFrame df.

In [ ]:
import numpy as np
import tensorflow as tf
from tqdm import tqdm

for index, row in tqdm(df.iterrows()):
  sentence  = row.recensione

  input_ids = tokenizer.encode(sentence, add_special_tokens=True, truncation=True, max_length=512)

  # Create tensor, use .cuda() to transfer the tensor to GPU
  tensor = torch.tensor(input_ids).long()
  # Fake batch dimension
  tensor = tensor.unsqueeze(0)

  # Call the model and get the logits
  logits = model(tensor)
  # Remove the fake batch dimension
  logits = logits.logits.squeeze(0)

  # The model was trained with a Log Likelyhood + Softmax combined loss, hence to extract probabilities we need a softmax on top of the logits tensor
  proba = nn.functional.softmax(logits, dim=0)

  # Unpack the tensor to obtain negative, neutral and positive probabilities
  negative, neutral, positive = proba

  df.loc[index,'sentiment_neg'] = proba[0].detach().numpy()
  df.loc[index,'sentiment_neut'] = proba[1].detach().numpy()
  df.loc[index,'sentiment_pos'] = proba[2].detach().numpy()


df['sentiment_pred'] = df[['sentiment_neg', 'sentiment_pos', 'sentiment_neut']].idxmax(axis=1)
print(df)

10044it [38:57,  4.30it/s]


                             id            POI rating   
0      66374440e18e1c186ce07e35   Palazzo Cesi      5  \
1      66374440e18e1c186ce07e35   Palazzo Cesi      5   
2      66374440e18e1c186ce07e35   Palazzo Cesi      4   
3      66374440e18e1c186ce07e35   Palazzo Cesi      4   
4      66374440e18e1c186ce07e35   Palazzo Cesi      4   
...                         ...            ...    ...   
10039  663c8b16ec1e0d04a559c75e   Cacio Pepe e      5   
10040  663c8b16ec1e0d04a559c75f  Annette Jolie      5   
10041  663c8b16ec1e0d04a559c75f  Annette Jolie      5   
10042  663c8b16ec1e0d04a559c760   Sorella Luna      5   
10043  663c8b16ec1e0d04a559c760   Sorella Luna      4   

                                              recensione  sentiment_neg   
0      Un palazzo appena restaurato, una ricca docume...       0.000708  \
1      Cinquecentesca dimora di una tra le famiglie i...       0.000184   
2      L'imponente palazzo in stile rinascimentale Ã¨...       0.000410   
3      Particol

In [ ]:
proba

tensor([0.2985, 0.0351, 0.6664], grad_fn=<SoftmaxBackward0>)

Il "tokenizer" è una componente fondamentale quando si lavora con modelli di linguaggio naturale (NLP). La sua funzione principale è quella di convertire il testo in una forma che il modello possa comprendere. Questo processo coinvolge solitamente la suddivisione del testo in parole o sotto-unità semantiche (come i token) e la successiva conversione di questi token in numeri, che possono essere utilizzati come input per il modello. Questo processo di tokenizzazione può anche includere altre operazioni, come la rimozione della punteggiatura, la gestione delle maiuscole e minuscole, e altro ancora.

Il "tensore", d'altra parte, è una struttura dati fondamentale nelle librerie di calcolo numerico, come TensorFlow o PyTorch, utilizzate per l'implementazione di modelli di machine learning e deep learning. In termini semplici, un tensore può essere considerato come un array multidimensionale, che può essere un vettore (1D), una matrice (2D) o una struttura multidimensionale di ordine superiore. I tensori sono essenziali poiché consentono di rappresentare i dati e le operazioni matematiche eseguite sui dati in modo efficiente e parallelo, specialmente quando si lavora con grandi quantità di dati.

In questo contesto il "tokenizer" viene utilizzato per convertire il testo delle recensioni in sequenze di token numerici, mentre il "tensore" viene utilizzato per rappresentare queste sequenze di token in una forma che il modello di machine learning (presumibilmente un modello di classificazione del sentiment) possa elaborare.

In [ ]:
df.head()


,id,POI,rating,recensione,sentiment_neg,sentiment_neut,sentiment_pos,sentiment_pred
0,66374440e18e1c186ce07e35,Palazzo Cesi,5,"Un palazzo appena restaurato, una ricca docume...",0.000708,0.028635,0.970657,sentiment_pos
1,66374440e18e1c186ce07e35,Palazzo Cesi,5,Cinquecentesca dimora di una tra le famiglie i...,0.000184,0.005011,0.994805,sentiment_pos
2,66374440e18e1c186ce07e35,Palazzo Cesi,4,L'imponente palazzo in stile rinascimentale Ã¨...,0.000410,0.001479,0.998111,sentiment_pos
3,66374440e18e1c186ce07e35,Palazzo Cesi,4,Particolari elogi vanno per il recupero e cons...,0.000697,0.003386,0.995917,sentiment_pos
4,66374440e18e1c186ce07e35,Palazzo Cesi,4,Palazzo Cesi di Acquasparta Ã© il principale m...,0.000538,0.054965,0.944497,sentiment_pos


In [ ]:
df.to_excel(r'C:\Users\franc\Desktop\PROJECTS\PROJECTS\RASTA\sentiment\POI_sentiment.xlsx', index=False)

In [ ]:
df = pd.read_excel(r'C:\Users\franc\Desktop\PROJECTS\PROJECTS\RASTA\sentiment\POI_sentiment.xlsx')
df.rating.value_counts()

rating
5    4966
4    2903
3    1350
1     474
2     351
Name: count, dtype: int64

In [ ]:
df.sentiment_pred.value_counts()

sentiment_pred
sentiment_pos     7967
sentiment_neut    1117
sentiment_neg      960
Name: count, dtype: int64

In [ ]:
df.to_csv(r"C:\Users\franc\Desktop\PROJECTS\PROJECTS\RASTA\sentiment\POI_sentiment.csv", index=False, sep='\t')

## Calcolo l'Accuracy

In [ ]:
df = pd.read_excel(r"C:\Users\franc\Desktop\PROJECTS\PROJECTS\RASTA\sentiment\POI_sentiment.xlsx")

In [ ]:
df

,id,POI,rating,recensione,sentiment_neg,sentiment_neut,sentiment_pos,sentiment_pred
0,66374440e18e1c186ce07e35,Palazzo Cesi,5,"Un palazzo appena restaurato, una ricca docume...",0.000708,0.028635,0.970657,sentiment_pos
1,66374440e18e1c186ce07e35,Palazzo Cesi,5,Cinquecentesca dimora di una tra le famiglie i...,0.000184,0.005011,0.994805,sentiment_pos
2,66374440e18e1c186ce07e35,Palazzo Cesi,4,L'imponente palazzo in stile rinascimentale Ã¨...,0.000410,0.001479,0.998111,sentiment_pos
3,66374440e18e1c186ce07e35,Palazzo Cesi,4,Particolari elogi vanno per il recupero e cons...,0.000697,0.003386,0.995917,sentiment_pos
4,66374440e18e1c186ce07e35,Palazzo Cesi,4,Palazzo Cesi di Acquasparta Ã© il principale m...,0.000538,0.054965,0.944497,sentiment_pos
...,...,...,...,...,...,...,...,...
10039,663c8b16ec1e0d04a559c75e,Cacio Pepe e,5,Ci siamo trovati di passaggio dopo aver visita...,0.000215,0.001434,0.998351,sentiment_pos
10040,663c8b16ec1e0d04a559c75f,Annette Jolie,5,"Girando per Assisi ,ho trovato un negozio fina...",0.000223,0.001533,0.998243,sentiment_pos
10041,663c8b16ec1e0d04a559c75f,Annette Jolie,5,A pochi passi da Piazza del Comune abbiamo sco...,0.000135,0.001645,0.998220,sentiment_pos
10042,663c8b16ec1e0d04a559c760,Sorella Luna,5,Negozio posto in una via secondaria ma dove va...,0.000398,0.003119,0.996483,sentiment_pos


In [ ]:
from sklearn.metrics import classification_report

df['GS']=0
df.loc[df['rating'] >= 4, 'GS']  = 'sentiment_pos'
df.loc[df['rating'] <= 2, 'GS']  = 'sentiment_neg'
df.loc[df['rating'] == 3, 'GS']  = 'sentiment_neut'
df

,id,POI,rating,recensione,sentiment_neg,sentiment_neut,sentiment_pos,sentiment_pred,GS
0,66374440e18e1c186ce07e35,Palazzo Cesi,5,"Un palazzo appena restaurato, una ricca docume...",0.000708,0.028635,0.970657,sentiment_pos,sentiment_pos
1,66374440e18e1c186ce07e35,Palazzo Cesi,5,Cinquecentesca dimora di una tra le famiglie i...,0.000184,0.005011,0.994805,sentiment_pos,sentiment_pos
2,66374440e18e1c186ce07e35,Palazzo Cesi,4,L'imponente palazzo in stile rinascimentale Ã¨...,0.000410,0.001479,0.998111,sentiment_pos,sentiment_pos
3,66374440e18e1c186ce07e35,Palazzo Cesi,4,Particolari elogi vanno per il recupero e cons...,0.000697,0.003386,0.995917,sentiment_pos,sentiment_pos
4,66374440e18e1c186ce07e35,Palazzo Cesi,4,Palazzo Cesi di Acquasparta Ã© il principale m...,0.000538,0.054965,0.944497,sentiment_pos,sentiment_pos
...,...,...,...,...,...,...,...,...,...
10039,663c8b16ec1e0d04a559c75e,Cacio Pepe e,5,Ci siamo trovati di passaggio dopo aver visita...,0.000215,0.001434,0.998351,sentiment_pos,sentiment_pos
10040,663c8b16ec1e0d04a559c75f,Annette Jolie,5,"Girando per Assisi ,ho trovato un negozio fina...",0.000223,0.001533,0.998243,sentiment_pos,sentiment_pos
10041,663c8b16ec1e0d04a559c75f,Annette Jolie,5,A pochi passi da Piazza del Comune abbiamo sco...,0.000135,0.001645,0.998220,sentiment_pos,sentiment_pos
10042,663c8b16ec1e0d04a559c760,Sorella Luna,5,Negozio posto in una via secondaria ma dove va...,0.000398,0.003119,0.996483,sentiment_pos,sentiment_pos


In [ ]:
print(classification_report(df['GS'], df['sentiment_pred']))

                precision    recall  f1-score   support

 sentiment_neg       0.62      0.72      0.67       825
sentiment_neut       0.31      0.26      0.28      1350
 sentiment_pos       0.88      0.89      0.89      7869

      accuracy                           0.79     10044
     macro avg       0.60      0.62      0.61     10044
  weighted avg       0.78      0.79      0.79     10044



In [ ]:
# Calcola il classification report
report = classification_report(df['GS'], df['sentiment_pred'], output_dict=True)

# Creare un DataFrame pandas dal classification report
df_report = pd.DataFrame(report).transpose()

# Salvare il DataFrame come file Excel
excel_filename = r"C:\Users\franc\Desktop\PROJECTS\PROJECTS\RASTA\sentiment\classification_report.xlsx"
df_report.to_excel(excel_filename, index=True)

# Sentiment - FEEL-IT Python Package

https://huggingface.co/MilaNLProc/feel-it-italian-sentiment?text=Mi+piaci.+Ti+amo


Il modello Feel-it-Italian-Sentiment esegue l'analisi del sentiment sull'italiano. Viene utilizzato il modello UmBERTo sul nuovo set di dati (cioè FEEL-IT) ottenendo prestazioni all'avanguardia su diversi corpus di riferimento.

I dati sono stati raccolti annotando tweet di un’ampia gamma di argomenti. In totale, ci sono 2037 tweet annotati con un'etichetta emozionale.

In [14]:
from transformers import pipeline

classifier = pipeline("text-classification", model='MilaNLProc/feel-it-italian-sentiment', top_k=2)
prediction = classifier("Oggi sono proprio contento!")
print(prediction)

c:\Users\franc\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


[[{'label': 'positive', 'score': 0.9997411370277405}, {'label': 'negative', 'score': 0.00025880170869641006}]]


In [15]:
with open(r"C:\Users\franc\Desktop\PROJECTS\PROJECTS\RASTA\POI con recensioni\RASTA.poi.json", 'r', encoding='latin-1') as file:
    data = json.load(file)

In [16]:
df = pd.DataFrame(columns=['id', 'POI', 'rating', 'recensione'])

# Append rows to the DataFrame
for a in data:
  id = a['_id']['$oid']
  poi = a['POI']

  for k in a.keys():
    if k != 'POI' and k!='_id':

      df = pd.concat([df, pd.DataFrame([{'id':id, 'POI': poi, 'rating':a[k]['rating'], 'recensione': a[k]['recensione']}])], ignore_index=True)

In [17]:
# Aggiungo al mio dataframe la oclonna relativa al sentiment predetto
df["sentiment_pred"] = {}  
df["sentiment_pos"] = {}
df["sentiment_neg"] = {}

In [18]:
from transformers import pipeline, AutoTokenizer

In [22]:
# Carica il tokenizer e la pipeline
tokenizer = AutoTokenizer.from_pretrained('MilaNLProc/feel-it-italian-sentiment')
classifier = pipeline("text-classification", model='MilaNLProc/feel-it-italian-sentiment', tokenizer=tokenizer, top_k=2)

# Funzione di tokenizzazione con padding e truncation
def preprocess(text, tokenizer, max_length=512):
    return tokenizer(text, padding='max_length', truncation=True, max_length=max_length, return_tensors='pt')

c:\Users\franc\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [20]:
for indice in df.values:
    # Estrai il testo da classificare
    text = indice[3]
    print(len(text))

194
272
1100
2356
1084
171
187
375
599
612
107
105
134
804
111
211
787
215
451
377
448
514
375
2242
232
153
275
414
171
718
198
289
572
481
102
221
163
149
167
312
898
301
625
571
1133
393
121
332
156
288
248
146
173
111
206
1010
147
771
144
178
200
378
457
415
756
144
255
503
234
162
698
230
178
459
435
175
232
1827
320
144
245
488
565
965
140
169
582
205
2090
877
111
193
262
330
515
341
2634
1273
712
238
335
569
105
373
516
197
2121
2353
368
414
792
248
198
174
200
2074
849
351
156
1105
278
601
751
1101
210
560
404
210
873
1521
218
409
255
2410
155
621
320
1239
238
688
240
218
277
406
242
155
253
1148
341
173
205
397
624
383
210
681
104
105
107
120
246
2997
560
313
337
201
775
1833
1585
673
399
523
1160
551
585
146
201
174
1724
140
193
232
2580
140
265
5344
166
2929
450
227
308
231
434
325
124
350
354
136
497
167
104
108
713
216
166
248
146
229
190
294
209
243
619
161
272
334
400
221
126
255
352
810
284
266
768
202
233
310
299
507
497
319
246
152
126
214
372
623
348
225
206
122
168
2

In [27]:
import pandas as pd

# Crea un nuovo DataFrame vuoto
df_excel = pd.DataFrame(columns=['id', 'POI', 'rating', 'recensione', 'sentiment_pos', 'sentiment_neg', 'sentiment_pred'])

# Funzione per classificare il testo e aggiornare il DataFrame
def classify_and_update(df, tokenizer, model):
    for i, row in df.iterrows():
        text = row['recensione']  
        
        # Tokenizza il testo e converte in tensori
        inputs = tokenizer(text, padding='max_length', truncation=True, max_length=512, return_tensors='pt')
        
        # Esegui la classificazione sul modello
        with torch.no_grad():
            outputs = model(**inputs)
            logits = outputs.logits
            probabilities = torch.softmax(logits, dim=1).squeeze()
            score_positive = probabilities[1].item()
            score_negative = probabilities[0].item()
        
        # Assegna i punteggi al DataFrame
        df.at[i, 'sentiment_pos'] = score_positive  
        df.at[i, 'sentiment_neg'] = score_negative  
        
        # Determina l'etichetta di sentiment
        sentiment = 'positive' if score_positive >= score_negative else 'negative'
        df.at[i, 'sentiment_pred'] = sentiment
        
        # Aggiungi la riga al DataFrame Excel
        df_excel.loc[i] = row
        
        # Salva il DataFrame Excel su file
        df_excel.to_excel('output.xlsx', index=False)
        
        num_rows = df.shape[0]
        print(f"Processed row {i+1}/{len(df)}")

# Esegui la classificazione e l'aggiornamento del DataFrame
classify_and_update(df, tokenizer, model)

# Stampa il dataframe per verificare i risultati
print(df)


Processed row 1/10044
Processed row 2/10044
Processed row 3/10044
Processed row 4/10044
Processed row 5/10044
Processed row 6/10044
Processed row 7/10044
Processed row 8/10044
Processed row 9/10044
Processed row 10/10044
Processed row 11/10044
Processed row 12/10044
Processed row 13/10044
Processed row 14/10044
Processed row 15/10044
Processed row 16/10044
Processed row 17/10044
Processed row 18/10044
Processed row 19/10044
Processed row 20/10044
Processed row 21/10044
Processed row 22/10044
Processed row 23/10044
Processed row 24/10044
Processed row 25/10044
Processed row 26/10044
Processed row 27/10044
Processed row 28/10044
Processed row 29/10044
Processed row 30/10044
Processed row 31/10044
Processed row 32/10044
Processed row 33/10044
Processed row 34/10044
Processed row 35/10044
Processed row 36/10044
Processed row 37/10044
Processed row 38/10044
Processed row 39/10044
Processed row 40/10044
Processed row 41/10044
Processed row 42/10044
Processed row 43/10044
Processed row 44/100

In [ ]:
df.to_excel(r'C:\Users\franc\Desktop\PROJECTS\PROJECTS\RASTA\sentiment\POI_sentiment_feel.xlsx')

In [ ]:
df = pd.read_excel(r'C:\Users\franc\Desktop\PROJECTS\PROJECTS\RASTA\sentiment\POI_sentiment_feel.xlsx')
df.rating.value_counts()

rating
5    4966
4    2903
3    1350
1     474
2     351
Name: count, dtype: int64